Importing the packages

In [26]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Subset, Dataset
from torchvision import datasets
from torchvision import transforms
import os
from os import listdir
from os.path import isfile, join
import cv2
from PIL import Image
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms.functional as fn


Importing and loading the data

In [ ]:
#!wget 'https://drive.google.com/uc?export=download&id=1Po-aUQtb-MYw92UhOYCN3gn1PP1uCHrH' -O 'ScientificPython.zip'

--2022-05-11 17:01:42--  https://drive.google.com/uc?export=download&id=1Po-aUQtb-MYw92UhOYCN3gn1PP1uCHrH
Resolving drive.google.com (drive.google.com)... 142.251.8.138, 142.251.8.102, 142.251.8.100, ...
Connecting to drive.google.com (drive.google.com)|142.251.8.138|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘ScientificPython.zip’

ScientificPython.zi     [ <=>                ]   2.17K  --.-KB/s    in 0s      

2022-05-11 17:01:43 (22.3 MB/s) - ‘ScientificPython.zip’ saved [2224]



In [2]:
!unzip 'ScientificPython.zip'

Archive:  ScientificPython.zip
   creating: ScientificPython/
   creating: ScientificPython/0/
  inflating: ScientificPython/0/11muscle.jpg  
  inflating: ScientificPython/0/bone1.jpg  
 extracting: ScientificPython/0/bone10.jpg  
  inflating: ScientificPython/0/bone11.jpg  
 extracting: ScientificPython/0/bone12.jpg  
  inflating: ScientificPython/0/bone13.jpg  
  inflating: ScientificPython/0/bone14.jpg  
  inflating: ScientificPython/0/bone15.jpg  
  inflating: ScientificPython/0/bone16.jpg  
 extracting: ScientificPython/0/bone2.jpg  
 extracting: ScientificPython/0/bone3.jpg  
 extracting: ScientificPython/0/bone4.jpg  
  inflating: ScientificPython/0/bone5.jpg  
 extracting: ScientificPython/0/bone6.jpg  
  inflating: ScientificPython/0/bone7.jpg  
  inflating: ScientificPython/0/bone8.jpg  
 extracting: ScientificPython/0/bone9.jpg  
 extracting: ScientificPython/0/fat07.jpg  
  inflating: ScientificPython/0/fat08.jpg  
 extracting: ScientificPython/0/fat09.jpg  
 extracting: Sc

Preprocess

In [27]:
preprocess = transforms.Compose(
    [transforms.ToTensor(), 
     transforms.Resize((128, 128))] #resizing every image to the same size
)

In [30]:
train_datasource = os.listdir(path = 'ScientificPython/')
train_datasource.sort() 
print(train_datasource) #printing the content in ScientificPython folder
train_datasource = train_datasource[:2] #we only need to first two folders as they contain the training data
print(train_datasource) #we are happy because only the first 2 folders remained
idx_neuron = 0 

train_img = []
train_cat = [] #category = 0 or 1 (non-neuron or neuron image) from the name of the forlder
for filename in train_datasource:
  current_filename = os.listdir('ScientificPython/' + filename) #we iterate through the folders 0 and 1
  current_filename.sort()
  for img_name in current_filename:
    img = cv2.imread(f'ScientificPython/{filename}/{img_name}') #we read the images from the folder
    img = preprocess(img) #applying the preprocess function
    train_img.append(img) 
    train_cat.append(filename) #filename will be the category/label
  idx_neuron += 1

train_img = torch.stack(train_img, dim=0)
train_cat = torch.stack(train_cat, dim = 0)
train_img.shape, train_cat.shape 

['0', '1', 'test']
['0', '1']


TypeError: ignored

In [20]:
class MyDataset(Dataset):
  def __init__(self, train_img, train_cat):
    self.data = train_img
    self.label = train_cat

  def __getitem__(self, index):
    x = self.data[index]
    y = self.label[index]
    return x, y

  def __len__(self):
    return len(self.data)

In [ ]:
train_dataset = MyDataset(train_img, train_cat) #the traning dataset consits of the image and the label
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True) #the training data will be loaded in random order

In [ ]:
#for elem, s in train_loader:
  #print(elem)
  #break

# The neural network

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_avaible() else "cpu")
print(device)

Training

In [ ]:
#the network: VGG16
Net = nn.Sequential(         
  nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, padding=1),
  nn.SELU(),
  #vggblock1
  nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1),
  nn.SELU(),
  nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1),
  nn.SELU(),
  #vggblock2
  nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
  nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
  nn.SELU(),
  nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1),
  nn.SELU(),
  #vggblock3
  nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
  nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1),
  nn.SELU(),
  nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1),
  nn.SELU(),
  nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1),
  nn.SELU(),
  #vggblock4
  nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
  nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, padding=1),
  nn.SELU(),
  nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1),
  nn.SELU(),
  nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1),
  nn.SELU(),
  #vggblock5
  nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
  nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1),
  nn.SELU(),
  nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1),
  nn.SELU(),
  nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1),
  nn.SELU(),

  nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
  ##classifier part
  nn.Flatten(),
  nn.Linear(in_features=8192, out_features=4096),
  nn.ReLU(),
  nn.Dropout(p=0.2),
  nn.Linear(in_features=4096, out_features=2048),
  nn.ReLU(),
  nn.Dropout(p=0.2),
  nn.Linear(in_features=2048, out_features=1024),
  nn.ReLU(),
  nn.Dropout(p=0.2),
  nn.Linear(in_features=1024, out_features=16), 
  nn.LogSoftmax(dim=1) #instead of this we can use CrossEntropyLoss
)

In [ ]:
Net = Net.to(device)

Optimizer + loss

In [ ]:
criterion = nn.NLLLoss()
#criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(Net.parameters(), lr=1e-4)
num_epochs = 50
Net.train()

#training:

for epoch in range(num_epochs):
  idx = 0
  for data, label in train_loader:
    idx += 1
    data = data.cuda()
    label = label.cuda()
    prediction = Net(data)
    loss = criterion(prediction, label)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if idx % 50 ==0:
      print(f"At iteration {idx} the current loss is: {float(loss)}")
    print(f"At epoch {idx} the current loss is: {float(loss)}")

In [ ]:
Net.eval()
data, label = dataset[0]
data.unsqueeze(dim=0).shape, label

In [ ]:
pred = Net(data.unsqueeze(dim=0).cuda())
pred.argmax(-1) == label

Testing

In [ ]:
#read in the test data
test_datasource = os.listdir(path = 'ScientificPython/test')
test_datasource.sort()
test_datasource = test_datasource[-1]
types = [str, int]

test_img = []

for img_name in test_datasource:
  img = cv2.imread(f'ScientificPython/test/{img_name}')
  img_test = preprocess(img)
  test_img.append(img_test)

test_img = torch.stack(test_img), dim=0)
#test_cat = torch.stack(test_cat, dim = 0)
test_img.shape, #test_cat.shape 

In [ ]:
test_dataset = MyDataset(test_img, test_cat)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
counter = 0
Net.eval()
for data, label in test_loader:
  prediction = Net(data.cuda())
  if prediction.argmax() == label.cuda():
    counter += 1

print(counter)
#print(counter/train_x.shape[0])

Results

In [ ]:
#printing the reaults
i = 0
for data, label in test_loader:
  data = data.cuda()
  label = label.cuda()

  output = Net(data.cuda())
  prediction = torch.argmax(output[0])

  print(file_names[i] + ' ' + str(prediction.item())
  i += 1